In [2]:
# Run this once
# (creates a realistic dataset with slight overlaps and label noise)
import pandas as pd
import numpy as np
import random

df = pd.read_csv("food_dataset_1000.csv")
df_mod = df.copy()

common_items = ["rice", "olive oil", "tomato", "bread", "egg", "milk", "nuts", "onion"]
for i in range(len(df_mod)):
    if random.random() < 0.1:
        df_mod.at[i, "ingredients"] += ", " + random.choice(common_items)

np.random.seed(42)
noise_idx = np.random.choice(df_mod.index, size=int(0.03 * len(df_mod)), replace=False)
df_mod.loc[noise_idx, "label"] = np.random.permutation(df_mod.loc[noise_idx, "label"])

neutral_recipes, neutral_labels = [], []
for _ in range(40):
    neutral_ing = ", ".join(np.random.choice(common_items, 5, replace=False))
    neutral_label = np.random.choice(["Weight Gain", "Weight Loss"])
    neutral_recipes.append(neutral_ing)
    neutral_labels.append(neutral_label)

neutral_df = pd.DataFrame({
    "ingredients": neutral_recipes,
    "label": neutral_labels,
    "calories": np.random.randint(200, 400, 40),
    "protein": np.random.randint(5, 20, 40),
    "carbs": np.random.randint(10, 50, 40),
    "fiber": np.random.randint(2, 8, 40),
    "fat": np.random.randint(2, 15, 40),
    "sugar": np.random.randint(0, 10, 40)
})

df_final = pd.concat([df_mod, neutral_df], ignore_index=True)
df_final = df_final.sample(frac=1, random_state=42).reset_index(drop=True)
df_final.to_csv("food_dataset_realistic.csv", index=False)
print("✅ Saved as 'food_dataset_realistic.csv'")


✅ Saved as 'food_dataset_realistic.csv'


In [3]:
df = pd.read_csv("food_dataset_realistic.csv")

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import pickle

vectorizer = TfidfVectorizer(
    stop_words='english',
    ngram_range=(1, 2),
    max_features=300,
    min_df=3,
    max_df=0.9
)
X = vectorizer.fit_transform(df['ingredients'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

model = LogisticRegression(C=0.2, max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = round(accuracy_score(y_test, y_pred) * 100, 2)
print("\n📊 Model Evaluation:")
print("Accuracy:", acc, "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))



📊 Model Evaluation:
Accuracy: 97.6 %

Classification Report:
               precision    recall  f1-score   support

 Weight Gain       0.97      0.98      0.98       104
 Weight Loss       0.98      0.97      0.98       104

    accuracy                           0.98       208
   macro avg       0.98      0.98      0.98       208
weighted avg       0.98      0.98      0.98       208



In [4]:
# =====================================
# 🔍 PREDICTION FUNCTION
# =====================================

def predict_food_type(ingredients):
    """
    Input: string of ingredients
    Output: Predicted label (Weight Gain / Weight Loss)
    """
    x_vec = vectorizer.transform([ingredients])
    prediction = model.predict(x_vec)[0]
    print(f"\n🧾 Ingredients: {ingredients}")
    print(f"💡 Prediction: {prediction}")

# Test few examples
examples = [
    "rolled oats, chia seeds, almond, no sugar added",
    "chocolate, milk solids, cocoa powder, sugar",
    "quinoa, flax seeds, spinach, olive oil",
    "potato, vegetable oil, salt, cheese powder",
    "white rice, butter, ghee, salt"
]

for ex in examples:
    predict_food_type(ex)



🧾 Ingredients: rolled oats, chia seeds, almond, no sugar added
💡 Prediction: Weight Loss

🧾 Ingredients: chocolate, milk solids, cocoa powder, sugar
💡 Prediction: Weight Gain

🧾 Ingredients: quinoa, flax seeds, spinach, olive oil
💡 Prediction: Weight Loss

🧾 Ingredients: potato, vegetable oil, salt, cheese powder
💡 Prediction: Weight Gain

🧾 Ingredients: white rice, butter, ghee, salt
💡 Prediction: Weight Gain


In [5]:
import pickle

with open("food_weight_model_final.pkl", "wb") as f:
    pickle.dump(model, f)
with open("tfidf_vectorizer_final.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("✅ Model and vectorizer saved successfully!")


✅ Model and vectorizer saved successfully!
